# K Nearest Neighbors Classifier

**Basic steps:**

1. Import the learning algorithm
2. Instantiate the model (choose hyper-parameters)
3. Learn the model
4. Predict the response

# Get Example Data

In [35]:
# target = InMichelin, whether or not a restaurant is in the Michelin guide
import pandas as pd
data = pd.read_csv("http://gattonweb.uky.edu/sheather/book/docs/datasets/MichelinNY.csv" , encoding="latin_1")
data.head()

,InMichelin,Restaurant Name,Food,Decor,Service,Price
0,0,14 Wall Street,19,20,19,50
1,0,212,17,17,16,43
2,0,26 Seats,23,17,21,35
3,1,44,19,23,16,52
4,0,A,23,12,19,24


In [4]:
# Delete extra variable that is not continuous
data = data.loc[:, data.columns != 'Restaurant Name']

data.head()

,InMichelin,Food,Decor,Service,Price
0,0,19,20,19,50
1,0,17,17,16,43
2,0,23,17,21,35
3,1,19,23,16,52
4,0,23,12,19,24


# Change variable names to X, y to create train/test split

In [6]:
y = data['InMichelin']
X = data.loc[:, data.columns != 'InMichelin']

print(y[0:5])
X.head()

0    0
1    0
2    0
3    1
4    0
Name: InMichelin, dtype: int64


,Food,Decor,Service,Price
0,19,20,19,50
1,17,17,16,43
2,23,17,21,35
3,19,23,16,52
4,23,12,19,24


# Train test split

In [8]:
from sklearn.model_selection import train_test_split

# Use train_test_split(X,y) to create four new data sets, defaults to .75/.25 split
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(X.shape)
X_train.shape

(164, 4)


(123, 4)

### Train model with k=5

In [10]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

#Print accuracy rounded to two digits to the right of decimal
print(knn.score(X_test, y_test))

y_pred = knn.predict(X_test) # y_pred includes your predictions

0.8292682926829268


### Train model with k=10

In [12]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

#Print accuracy rounded to two digits to the right of decimal
print(f'accuracy: {str(knn.score(X_test, y_test))}')
y_pred = knn.predict(X_test)

accuracy: 0.8292682926829268


In [13]:
y_pred # view predictions for test data

array([0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0])

## Using Cross validation for model evaluation

In [15]:
#import cross validation functions from sk learn

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold

from sklearn.model_selection import KFold
import numpy as np
from statistics import mean

# Set up function parameters for different cross validation strategies
kfold = KFold(n_splits=5)
skfold = StratifiedKFold(n_splits=5, shuffle=True)
rkf = RepeatedKFold(n_splits=5, n_repeats=10)

print("KFold:\n{}".format(
mean(cross_val_score(KNeighborsClassifier(n_neighbors=5), X_train, y_train, cv=kfold))))

print("\nStratifiedKFold:\n{}".format(
mean(cross_val_score(KNeighborsClassifier(n_neighbors=5), X_train, y_train, cv=skfold))))

print("\nRepeatedKFold:\n{}".format(
mean(cross_val_score(KNeighborsClassifier(n_neighbors=5), X_train, y_train, cv=rkf))))

KFold:
0.8049999999999999

StratifiedKFold:
0.7973333333333333

RepeatedKFold:
0.7886666666666666


## Tuning models with grid search

In [17]:
from sklearn.model_selection import GridSearchCV
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

#create dictionary data object with keys equal to parameter name 'n_neighbors'
#for knn model and values equal to range of k values to create models for

param_grid = {'n_neighbors': np.arange(1, 15, 2)} #np.arange creates sequence of numbers for each k value

grid = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=10)

#use model methods to fit score and predict model:
grid.fit(X_train, y_train)

#extract best score and parameter by calling objects "best_score_" and "best_params_"
print("best mean cross-validation score: {:.1f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))
print("test-set score: {:.3f}".format(grid.score(X_test, y_test)))

best mean cross-validation score: 0.8
best parameters: {'n_neighbors': 5}
test-set score: 0.829


In [18]:
# view data with complete tuning results
results = pd.DataFrame(grid.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000440,0.000114,0.000769,0.000236,1,{'n_neighbors': 1},0.692308,0.615385,0.538462,0.916667,0.666667,0.750000,0.833333,0.750000,0.833333,0.750000,0.734615,0.106039,7
1,0.000331,0.000003,0.000609,0.000032,3,{'n_neighbors': 3},0.615385,0.615385,0.692308,0.916667,0.750000,0.750000,0.750000,0.833333,0.750000,1.000000,0.767308,0.115671,6
2,0.000331,0.000005,0.000597,0.000012,5,{'n_neighbors': 5},0.615385,0.615385,0.692308,0.916667,0.750000,0.916667,0.750000,1.000000,0.916667,0.916667,0.808974,0.133567,1
3,0.000327,0.000003,0.000597,0.000020,7,{'n_neighbors': 7},0.538462,0.615385,0.615385,0.916667,0.750000,0.833333,0.833333,1.000000,0.916667,0.916667,0.793590,0.148900,5
4,0.000325,0.000002,0.000590,0.000009,9,{'n_neighbors': 9},0.538462,0.692308,0.615385,0.916667,0.750000,0.833333,0.833333,1.000000,0.916667,0.916667,0.801282,0.141288,4
5,0.000342,0.000018,0.000634,0.000024,11,{'n_neighbors': 11},0.615385,0.692308,0.615385,0.916667,0.750000,0.833333,0.833333,1.000000,0.916667,0.916667,0.808974,0.128263,1
6,0.000329,0.000010,0.000590,0.000008,13,{'n_neighbors': 13},0.615385,0.692308,0.615385,0.916667,0.750000,0.833333,0.833333,1.000000,0.916667,0.916667,0.808974,0.128263,1
